In [36]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import re
import encodings

In [26]:
number_pattern = re.compile('\w*\d+\w*')
article_pattern = re.compile('\w*[d+,-.]\w+')

In [27]:
df = pd.read_csv("/home/rsolomatin/metalurgi/companies_info_v2.csv", index_col=0)
df["ИНН"] = df["ИНН"].astype(str)

In [28]:
df.head()

,Индекс,Название,ИНН,ОГРН,Путь,ВК,Статус,Время
0,1,"ЕВРОКОЛОР, ООО",352526947648.0,1153525047092,"Sber_parser/1-101/data/ЕВРОКОЛОР, ООО_0",NaN,-1,0.000000
1,2,"ВЮМ, ООО",772503503336.0,1147746090887,"Sber_parser/1-101/data/ВЮМ, ООО_0",NaN,1,1.529426
2,3,"НОПРОФ, ООО",781110795023.0,1037825017340,"Sber_parser/1-101/data/НОПРОФ, ООО_0",NaN,1,5.654249
3,4,"СТАЙЕР, ООО МПК",741111765876.0,1117411000882,"Sber_parser/1-101/data/СТАЙЕР, ООО МПК_0",NaN,-1,0.000000
4,5,"ЗАМОК, ООО",590400687580.0,1105904005833,"Sber_parser/1-101/data/ЗАМОК, ООО_0",NaN,-1,0.000000


In [5]:
%%time

path = "/home/rsolomatin/metalurgi/Sber_parser"
path_inn = {path:inn for path, inn in zip(df["Путь"], df["ИНН"])}
data = []
for parent_directory in (os.listdir(path)):
    main_dir = os.path.join(path, f"{parent_directory}/data")
    for directory in tqdm(os.listdir(main_dir), desc=parent_directory):
        df_path = (os.path.join("Sber_parser", parent_directory, "data", directory))
        inn = path_inn.get(df_path, None)
        subdir = os.path.join(main_dir, directory)
        for file in os.listdir(subdir):
            file_path = os.path.join(subdir, file)
            if not os.path.isfile(file_path):
                continue
            with open(file_path) as f:
                lines = f.readlines()
                for i, line in enumerate(lines):
                    words = line.split()
                    if len(words) <= 3:
                        continue
                    final_line = []
                    for word in words:
                        word = number_pattern.sub(r'', word)
                        word = article_pattern.sub(r'', word)
                        final_line.append(word)
                    data.append({"path":file_path, "inn": inn, "sent_num":i, "line": " ".join(final_line)})

3801-4001: 100%|██████████| 93/93 [00:37<00:00,  2.45it/s]

CPU times: user 12min 20s, sys: 16.6 s, total: 12min 37s
Wall time: 15min 22s


In [6]:
len(data)

27415533

In [7]:
%%time
pd.DataFrame(data).to_csv("parsed_metalurgs.csv", index=False)

CPU times: user 2min 5s, sys: 8.7 s, total: 2min 14s
Wall time: 3min 2s


In [5]:
path = "/home/rsolomatin/metalurgi/Sber_parser"
path_inn = {path:inn for path, inn in zip(df["Путь"], df["ИНН"])}
for parent_directory in (os.listdir(path)):
    main_dir = os.path.join(path, f"{parent_directory}/data")
    for directory in tqdm(os.listdir(main_dir), desc=parent_directory):
        df_path = (os.path.join("Sber_parser", parent_directory, "data", directory))
        inn = path_inn[df_path]
        

3801-4001: 100%|██████████| 93/93 [00:00<00:00, 129763.90it/s]


In [15]:
%%time
test = pd.read_csv("parsed_metalurgs.csv")

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27415533 entries, 0 to 27415532
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   path      object 
 1   inn       float64
 2   sent_num  int64  
 3   line      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 836.7+ MB


In [17]:
test.describe()

,inn,sent_num
count,2.698177e+07,2.741553e+07
mean,5.679129e+11,4.974739e+02
std,2.036375e+11,4.010993e+03
min,1.030096e+10,0.000000e+00
25%,4.704210e+11,8.200000e+01
50%,6.164778e+11,1.900000e+02
75%,7.709745e+11,4.580000e+02
max,9.108065e+11,2.850930e+05


In [22]:
test[]["path"].iloc[0]

'/home/rsolomatin/metalurgi/Sber_parser/6401-6601/data/ОМГ РЫБИНСК, ООО_1077610004108_0/page_4.txt'

In [23]:
test["inn"].isna().sum()

433764

In [24]:
test["inn"].shape

(27415533,)

In [39]:
path = "/home/rsolomatin/metalurgi/Sber_parser"
path_inn = {path:inn for path, inn in zip(df["Путь"], df["ИНН"])}
data = []
for parent_directory in (os.listdir(path)):
    main_dir = os.path.join(path, f"{parent_directory}/data")
    for directory in tqdm(os.listdir(main_dir), desc=parent_directory):
        df_path = (os.path.join("Sber_parser", parent_directory, "data", directory))
        inn = path_inn[df_path]
        # print(inn, type(inn))
        if inn == "nan":
            print(df_path)

6401-6601: 100%|██████████| 71/71 [00:00<00:00, 327247.89it/s]


Sber_parser/6401-6601/data/ОМГ РЫБИНСК, ООО_1077610004108_0


4001-4201: 100%|██████████| 88/88 [00:00<00:00, 300813.98it/s]


Sber_parser/4001-4201/data/УРАЛЭЛЕКТРОМЕДЬ, АО_1026600726657_0
Sber_parser/4001-4201/data/УРАЛЭЛЕКТРОМЕДЬ, АО_1026600726657_1
Sber_parser/4001-4201/data/КЗОЦМ, АО_1024301313376_0
Sber_parser/4001-4201/data/СВЯТОГОР, АО_1026601213980_0


3401-3601: 100%|██████████| 102/102 [00:00<00:00, 253147.34it/s]


...

Sber_parser/5801-6001/data/МИТЕК ИНДАСТРИС РУ, ООО_1086672026429_0
Sber_parser/5801-6001/data/ПИК-ПРОДАКШН, ООО_1135007001800_0


4601-4801: 100%|██████████| 94/94 [00:00<00:00, 180276.44it/s]


Sber_parser/4601-4801/data/РУСАЛ САЯНАЛ, АО_1021900673442_0
Sber_parser/4601-4801/data/РУСАЛ КРАСНОЯРСК, АО_1022402468010_0
Sber_parser/4601-4801/data/РУСАЛ НОВОКУЗНЕЦК, АО_1024201821093_0
Sber_parser/4601-4801/data/АГМК, ООО_1072706000421_0
Sber_parser/4601-4801/data/УРАЛЬСКАЯ ФОЛЬГА, АО_1036602271386_0
Sber_parser/4601-4801/data/РУСАЛ САЯНОГОРСК, АО_1021900670758_0
Sber_parser/4601-4801/data/РУСАЛ БРАТСК, ПАО_1023800836377_0
Sber_parser/4601-4801/data/РУСАЛ АЧИНСК, АО_1022401155325_0
Sber_parser/4601-4801/data/ТГМК, ООО_1172724025286_0


3801-4001: 100%|██████████| 93/93 [00:00<00:00, 191492.52it/s]

Sber_parser/3801-4001/data/ПФ КМТ, ОАО_1027808912890_0
